Вы получили основные навыки обработки данных, теперь пора испытать их на практике. Сейчас вам предстоит заняться задачей классификации.

Представлен датасет центра приюта животных, и вашей задачей будет обучить модель таким образом, чтобы  по определенным признакам была возможность максимально уверенно предсказать метки 'Adoption' и 'Transfer' (столбец “outcome_type”).

Здесь вы вольны делать что угодно. Я хочу видеть от вас:
1. Проверка наличия/обработка пропусков
2. Проверьте взаимосвязи между признаками
3. Попробуйте создать свои признаки
4. Удалите лишние
5. Обратите внимание на текстовые столбцы. Подумайте, что можно извлечь полезного оттуда
6. Использование профайлера вам поможет.
7. Не забывайте, что у вас есть PCA (Метод главных компонент). Он может пригодиться.

Вспомните о всем, что я говорил на предыдущих занятиях. Не все будет пригодится, но в жизни вам никто не будет говорить, что использовать :)

Хорошим классификатором для этой задачи будет "Случайный лес" (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Понимать суть работы "леса" не обязательно на данном этапе, но качество предсказаний будет выше, чем с линейным классификатором. (если желаете, вот гайд https://adataanalyst.com/scikit-learn/linear-classification-method/)

Желаю успеха :)

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from datetime import datetime
from datetime import time
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = pd.read_csv('aac_shelter_outcomes.csv')
data.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


Очевидно что это задача классификации:
Порядок действий:
1. Описываем признаки+
2. Смотрим Nan-ы и дубликаты (удалено 10), решаем что с ними делать.+

    2.5. Превратить категории в вещественные признаки (цифры)(preprocessing.LabelEncoder())
    clear_data['breed'] = l_encoding.fit_transform(breed.animal_type) (СДЕЛАНО)


3. Выбираем полезные признаки(столбцы) + создаём новые (Вделить из datetime год и месяц рождения)
(пропустил)4. Изучаем аномалии что то с ними делаем ( Данные категориальные- при дальнейшем изучении данных отверг подход)
(пропустил)5. Изучаем выбросы и что то с ними делаем ( Данные категориальные- при дальнейшем изучении данных отверг подход)
(пропустил)6. По признакам похоже что ,без one-hot encoding никак( Нет при дальнейшем изучении данных отверг подход)
7. Изучаем RandomForest и  чем его кормить (подсмотрел но скормил)
8. Прыгаем на граблях...(При переводе из строки промежутка времени("2 weeks") появились Nan-ы, немного, поэтому удалил.
9. Пьём шампанское если всё получилось. Неожиданно....99%!!!!!!!



In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78256 entries, 0 to 78255
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   age_upon_outcome  78248 non-null  object
 1   animal_id         78256 non-null  object
 2   animal_type       78256 non-null  object
 3   breed             78256 non-null  object
 4   color             78256 non-null  object
 5   date_of_birth     78256 non-null  object
 6   datetime          78256 non-null  object
 7   monthyear         78256 non-null  object
 8   name              54370 non-null  object
 9   outcome_subtype   35963 non-null  object
 10  outcome_type      78244 non-null  object
 11  sex_upon_outcome  78254 non-null  object
dtypes: object(12)
memory usage: 7.2+ MB


age_upon_outcome- время проведённое в приюте.
                10 Nan-ов - можно и удалить (УДАЛЕНО)
                46 - уникальных значений
                -Придумать как обработать текст- перевести в date.time или иной формат для анализа зависимости возраста поступления с типом выбытия из приюта. !!! создать новый признак на основании текстового поля возраст в днях= если в поле содержит int '1'+str "week", то real_age = 1*7   (но что делать с '0 years')
                
animal_id- идентификационный номер в некой БД- пользы пока нет- можно и удалить (СТОЛБЕЦ УДАЛЕН)
                   Nan-ов нет
                   
animal_type - вид животного обязательно оставить, 
                Nan-ов нет
                5-уникальных значений,

breed- порода животного. как использовать пока не решил.
                Nan-ов нет
                2128-уникальных значений, 

color- окрас животного, скорее всего прийдётся делать one-hot-encoding
                Nan-ов нет
                525-уникальных значений, теперь one-hot-encoding под вопросом...или кластеризация??
                
                
date_of_birth- дата рождения (СТОЛБЕЦ УДАЛЕН)
                Nan-ов нет
datetime- тоже но в datetime для классификации создать столбец год, столбец месяц и столбец UNIXdate(Хотя по unix не уверен это больше для регрессии).
                Nan-ов нет
monthyear- тоже но в datetime (СТОЛБЕЦ УДАЛЕН)
                Nan-ов нет

name- кличка животного, Nan-ов 23884, пока не понимаю как применить кличку... если бы  не большое количество Nan-ов, то возможно по наиболие часто встречаемому имени у конкретного типа животного можно бы было что-то предсказать( "Барсик- скорее Кот а не Рыбка") 
но возможно ли предсказать как он покинул приют!!!!-  сначала поменять Nan на John Doe, а затем скорее всего удалить (СТОЛБЕЦ УДАЛЕН)

outcome_subtype Причина "выбытия" из приюта непонятный признак. 
                очень много Nan-ов скорее всего необходимо менять на Unknown или восстановить по outcome_type
                19 - уникальных значений
sex_upon_outcome - гендерный пол??? до выхода из приюта. Nan-ов- 2 штуки- drop-нуть (УДАЛЕНО) или восстановить по outcome_subtype
                5 - уникальных значений

outcome_type- Тип "выбытия" из приюта. Nan-ов- 12 штук - drop-нуть (УДАЛЕНО) или восстановить по outcome_subtype
                9 - уникальных значений

In [34]:
clear_data= data.drop_duplicates()
clear_data.dropna(subset = ['age_upon_outcome','sex_upon_outcome','outcome_type'], inplace = True)
clear_data = clear_data[['age_upon_outcome','animal_type','breed','color','datetime','outcome_subtype','sex_upon_outcome','outcome_type']]
clear_data= clear_data.fillna('unknown')
print(clear_data.info())
clear_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78227 entries, 0 to 78255
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   age_upon_outcome  78227 non-null  object
 1   animal_type       78227 non-null  object
 2   breed             78227 non-null  object
 3   color             78227 non-null  object
 4   datetime          78227 non-null  object
 5   outcome_subtype   78227 non-null  object
 6   sex_upon_outcome  78227 non-null  object
 7   outcome_type      78227 non-null  object
dtypes: object(8)
memory usage: 5.4+ MB
None


C:\Users\Prozo\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,age_upon_outcome,animal_type,breed,color,datetime,outcome_subtype,sex_upon_outcome,outcome_type
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-22T16:04:00,Partner,Intact Male,Transfer
1,1 year,Dog,Beagle Mix,White/Brown,2013-11-07T11:47:00,Partner,Spayed Female,Transfer
2,1 year,Dog,Pit Bull,Blue/White,2014-06-03T14:20:00,unknown,Neutered Male,Adoption
3,9 years,Dog,Miniature Schnauzer Mix,White,2014-06-15T15:50:00,Partner,Neutered Male,Transfer
4,5 months,Other,Bat Mix,Brown,2014-07-07T14:04:00,Rabies Risk,Unknown,Euthanasia


2.5. Превратить категории в вещественные признаки (цифры)(preprocessing.LabelEncoder()) clear_data['breed'] = l_encoding.fit_transform(breed.animal_type) (СДЕЛАНО)

In [35]:
# clear_data['animal_type'] = animal_type.fit_transform(clear_data.animal_type)
l_coder = preprocessing.LabelEncoder()
clear_data['animal_type'] = l_coder.fit_transform(clear_data.animal_type)
clear_data['breed'] = l_coder.fit_transform(clear_data.breed)
clear_data['color'] = l_coder.fit_transform(clear_data.color)
clear_data['outcome_subtype'] = l_coder.fit_transform(clear_data.outcome_subtype)
clear_data['sex_upon_outcome'] = l_coder.fit_transform(clear_data.sex_upon_outcome)
clear_data['outcome_type'] = l_coder.fit_transform(clear_data.outcome_type)
clear_data.head()

,age_upon_outcome,animal_type,breed,color,datetime,outcome_subtype,sex_upon_outcome,outcome_type
0,2 weeks,1,937,321,2014-07-22T16:04:00,12,1,8
1,1 year,2,255,474,2013-11-07T11:47:00,12,3,8
2,1 year,2,1612,119,2014-06-03T14:20:00,19,2,0
3,9 years,2,1467,463,2014-06-15T15:50:00,12,2,8
4,5 months,4,253,122,2014-07-07T14:04:00,14,4,3


Попытаемся выделить из ['datetime'] значение ['month'] и ['year']

In [36]:
dt_df= pd.to_datetime(clear_data['datetime'])
clear_data['month'] = dt_df.dt.month
clear_data['year'] = dt_df.dt.year
clear_data.head()

,age_upon_outcome,animal_type,breed,color,datetime,outcome_subtype,sex_upon_outcome,outcome_type,month,year
0,2 weeks,1,937,321,2014-07-22T16:04:00,12,1,8,7,2014
1,1 year,2,255,474,2013-11-07T11:47:00,12,3,8,11,2013
2,1 year,2,1612,119,2014-06-03T14:20:00,19,2,0,6,2014
3,9 years,2,1467,463,2014-06-15T15:50:00,12,2,8,6,2014
4,5 months,4,253,122,2014-07-07T14:04:00,14,4,3,7,2014


Остался один столбец с категориальными данными ['age_upon_outcome'] необходимо как то его привести к вещественным данным- 46 уникальных значений
Нужно поломать голову с RegEx-ами

In [37]:
clear_data.age_upon_outcome.unique()

array(['2 weeks', '1 year', '9 years', '5 months', '4 months', '3 years',
       '1 month', '3 months', '2 years', '2 months', '4 years', '8 years',
       '3 weeks', '8 months', '12 years', '7 years', '5 years', '6 years',
       '5 days', '10 months', '4 weeks', '10 years', '2 days', '6 months',
       '14 years', '11 months', '15 years', '7 months', '13 years',
       '11 years', '16 years', '9 months', '3 days', '6 days', '4 days',
       '5 weeks', '1 week', '1 day', '1 weeks', '0 years', '17 years',
       '20 years', '18 years', '19 years', '22 years', '25 years'],
      dtype=object)

In [38]:
# Подсмотрел алгоритм чтобы не лезть в RegEx, переводим текстовые даты в годы. Очень долго считает(

def age_count (row):
    
      if row['age_upon_outcome'] == "0 years" :
        return 0
      if row['age_upon_outcome'] == "1 year" :
        return 1
      if row['age_upon_outcome'] == "2 years" : 
        return 2
      if row['age_upon_outcome'] == "3 years" : 
        return 3
      if row['age_upon_outcome'] == "4 years" : 
        return 4
      if row['age_upon_outcome'] == "5 years" : 
        return 5
      if row['age_upon_outcome'] == "6 years" : 
        return 6
      if row['age_upon_outcome'] == "7 years" : 
        return 7
      if row['age_upon_outcome'] == "8 years" : 
        return 8
      if row['age_upon_outcome'] == "9 years" : 
        return 9
      if row['age_upon_outcome'] == "10 years" : 
        return 10
      if row['age_upon_outcome'] == "11 years" : 
        return 11
      if row['age_upon_outcome'] == "12 years" : 
        return 12
      if row['age_upon_outcome'] == "13 years" : 
        return 13
      if row['age_upon_outcome'] == "14 years" : 
        return 14
      if row['age_upon_outcome'] == "15 years" : 
        return 15
      if row['age_upon_outcome'] == "16 years" :
        return 16
      if row['age_upon_outcome'] == "17 years" :
        return 17
      if row['age_upon_outcome'] == "18 years" :
        return 18
      if row['age_upon_outcome'] == "20 years" :
        return 20
      if row['age_upon_outcome'] == "1 month" :
        return 1/12
      if row['age_upon_outcome'] == "2 months" :
        return 2/12
      if row['age_upon_outcome'] == "3 months" :
        return 3/12
      if row['age_upon_outcome'] == "4 months" :
        return 4/12
      if row['age_upon_outcome'] == "5 months" :
        return 5/12
      if row['age_upon_outcome'] == "6 months" :
        return 6/12
      if row['age_upon_outcome'] == "7 months" :
        return 7/12
      if row['age_upon_outcome'] == "8 months" :
        return 8/12
      if row['age_upon_outcome'] == "9 months" :
        return 9/12
      if row['age_upon_outcome'] == "10 months" :
        return 10/12
      if row['age_upon_outcome'] == "11 months" :
        return 11/12
      if row['age_upon_outcome'] == "1 week" :
        return 1/365*7
      if row['age_upon_outcome'] == "1 weeks" :
        return 1/365*7
      if row['age_upon_outcome'] == "2 weeks" :
        return 2/365*7
      if row['age_upon_outcome'] == "3 weeks" :
        return 3/365*7
      if row['age_upon_outcome'] == "4 weeks" :
        return 4/365*7
      if row['age_upon_outcome'] == "5 weeks" :
        return 5/365*7
      if row['age_upon_outcome'] == "1 day" :
        return 1/365
      if row['age_upon_outcome'] == "2 days" :
        return 2/365
      if row['age_upon_outcome'] == "3 days" :
        return 3/365
      if row['age_upon_outcome'] == "4 days" :
        return 4/365
      if row['age_upon_outcome'] == "5 days" :
        return 5/365
      if row['age_upon_outcome'] == "6 days" :
        return 6/365
%time    
clear_data['age']=clear_data.apply(age_count, axis=1).round(3)
clear_data

Wall time: 0 ns


,age_upon_outcome,animal_type,breed,color,datetime,outcome_subtype,sex_upon_outcome,outcome_type,month,year,age
0,2 weeks,1,937,321,2014-07-22T16:04:00,12,1,8,7,2014,0.038
1,1 year,2,255,474,2013-11-07T11:47:00,12,3,8,11,2013,1.000
2,1 year,2,1612,119,2014-06-03T14:20:00,19,2,0,6,2014,1.000
3,9 years,2,1467,463,2014-06-15T15:50:00,12,2,8,6,2014,9.000
4,5 months,4,253,122,2014-07-07T14:04:00,14,4,3,7,2014,0.417
...,...,...,...,...,...,...,...,...,...,...,...
78250,1 month,2,1106,182,2018-02-01T18:40:00,6,2,0,2,2018,0.083
78251,1 month,2,1106,182,2018-02-01T18:26:00,6,3,0,2,2018,0.083
78252,3 years,2,1413,119,2018-02-01T18:06:00,19,2,0,2,2018,3.000
78254,2 months,2,1991,339,2018-02-01T18:32:00,19,3,0,2,2018,0.167


In [39]:
#убираем столбца с категориальными данными [age_upon_outcome] и [datetime]

clear_data= clear_data.drop(['age_upon_outcome','datetime'],axis=1)

clear_data.head()

,animal_type,breed,color,outcome_subtype,sex_upon_outcome,outcome_type,month,year,age
0,1,937,321,12,1,8,7,2014,0.038
1,2,255,474,12,3,8,11,2013,1.000
2,2,1612,119,19,2,0,6,2014,1.000
3,2,1467,463,12,2,8,6,2014,9.000
4,4,253,122,14,4,3,7,2014,0.417


In [40]:
#При преобразовании появилось 19 Nan ов, удалим их.
clear_data.dropna(subset = ['age'], inplace = True)

In [43]:
clear_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78209 entries, 0 to 78255
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   animal_type       78209 non-null  int32  
 1   breed             78209 non-null  int32  
 2   color             78209 non-null  int32  
 3   outcome_subtype   78209 non-null  int32  
 4   sex_upon_outcome  78209 non-null  int32  
 5   outcome_type      78209 non-null  int32  
 6   month             78209 non-null  int64  
 7   year              78209 non-null  int64  
 8   age               78209 non-null  float64
dtypes: float64(1), int32(6), int64(2)
memory usage: 4.2 MB


Попробуем запустить RandomForest

In [44]:
# Делим на тренировочную и тестовую части... прогнозируемый параметр ['outcome_type']
#(данные категорий приведены к вещественным показателям)
ytrain = clear_data['outcome_type']
xtrain = clear_data.drop('outcome_type', axis=1)

In [45]:
ytrain.head()

0    8
1    8
2    0
3    8
4    3
Name: outcome_type, dtype: int32

In [46]:
xtrain.head()

,animal_type,breed,color,outcome_subtype,sex_upon_outcome,month,year,age
0,1,937,321,12,1,7,2014,0.038
1,2,255,474,12,3,11,2013,1.000
2,2,1612,119,19,2,6,2014,1.000
3,2,1467,463,12,2,6,2014,9.000
4,4,253,122,14,4,7,2014,0.417


In [47]:
# Точность предсказания---не работает т.к. есть категориальные данные(str) в столбце ['age_upon_outcome']
#пишет что большие цифры или Nanы, 
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(xtrain, ytrain)
tra_score=rf.score(xtrain, ytrain)

In [48]:
print('Training acc:', round(tra_score*100, 2), '%')

Training acc: 99.04 %
